In [1]:
#Import dependencies
import os
import pandas as pd



spark_version = 'spark-3.5.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:4 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ Packages [53.5 kB]
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:6 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [917 kB]
Hit:8 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Get:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease [18.1 kB]
Get:10 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:11 http://security.ubuntu.com/ubuntu jammy-security/universe amd64 Packages [1,088 kB]
Get:12 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease [24.3 kB

In [2]:
# Create a SparkSession
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
#Read in csv into pandas
players_df = pd.read_csv(('/content/drive/MyDrive/Colab Notebooks/Project-4/Resources/CollegeBasketballPlayers2009-2021.csv'))

# display the DataFrame
players_df

<ipython-input-5-4c52abb59c2b>:2: DtypeWarning: Columns (27) have mixed types. Specify dtype option on import or set low_memory=False.
  players_df = pd.read_csv(('/content/drive/MyDrive/Colab Notebooks/Project-4/Resources/CollegeBasketballPlayers2009-2021.csv'))


,player_name,team,conf,GP,Min_per,Ortg,usg,eFG,TS_per,ORB_per,...,dgbpm,oreb,dreb,treb,ast,stl,blk,pts,Unnamed: 64,Unnamed: 65
0,DeAndrae Ross,South Alabama,SB,26,29.5,97.3,16.6,42.5,44.43,1.6,...,-1.941150,0.1923,0.6154,0.8077,1.1923,0.3462,0.0385,3.8846,NaN,6.22026
1,Pooh Williams,Utah St.,WAC,34,60.9,108.3,14.9,52.4,54.48,3.8,...,-0.247934,0.6765,1.2647,1.9412,1.8235,0.4118,0.2353,5.9412,NaN,3.94375
2,Jesus Verdejo,South Florida,BE,27,72.0,96.2,21.8,45.7,47.98,2.1,...,-0.883163,0.6296,2.3333,2.9630,1.9630,0.4815,0.0000,12.1852,NaN,10.92680
3,Mike Hornbuckle,Pepperdine,WCC,30,44.5,97.7,16.0,53.6,53.69,4.1,...,-0.393459,0.7000,1.4333,2.1333,1.1000,0.5667,0.1333,4.9333,NaN,6.77427
4,Anthony Brown,Pacific,BW,33,56.2,96.5,22.0,52.8,54.31,8.3,...,-0.668318,1.4242,3.3030,4.7273,0.8485,0.4545,0.3333,7.5758,NaN,0.00000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61056,Trey Patterson,Villanova,BE,2,0.3,60.5,19.3,0.0,0.00,0.0,...,16.018200,0.0000,0.0000,0.0000,0.5000,0.5000,0.0000,0.0000,Pure PG,0.00000
61057,Stavros Polatoglou,Northwestern St.,Slnd,4,1.3,28.3,7.1,0.0,0.00,7.0,...,-4.993820,0.2500,0.0000,0.2500,0.0000,0.0000,0.0000,0.0000,C,0.00000
61058,Sandy Ryan,Tulane,Amer,1,0.1,0.0,0.0,0.0,0.00,0.0,...,-1.126810,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,PF/C,0.00000
61059,Ty Larson,Texas Tech,B12,1,0.1,0.0,0.0,0.0,0.00,0.0,...,-2.380750,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,PF/C,0.00000


In [6]:
#Dropping columns that have too many null values
cleaned_players_df = players_df.drop(columns=["num", "Rec Rank", "ast/tov", "rimmade", "rimmade+rimmiss", "midmade", "midmade+midmiss", "rimmade/(rimmade+rimmiss)",
                                               "midmade/(midmade+midmiss)", "dunksmade", "dunksmiss+dunksmade", "dunksmade/(dunksmade+dunksmiss)"])
cleaned_players_df

,player_name,team,conf,GP,Min_per,Ortg,usg,eFG,TS_per,ORB_per,...,dgbpm,oreb,dreb,treb,ast,stl,blk,pts,Unnamed: 64,Unnamed: 65
0,DeAndrae Ross,South Alabama,SB,26,29.5,97.3,16.6,42.5,44.43,1.6,...,-1.941150,0.1923,0.6154,0.8077,1.1923,0.3462,0.0385,3.8846,NaN,6.22026
1,Pooh Williams,Utah St.,WAC,34,60.9,108.3,14.9,52.4,54.48,3.8,...,-0.247934,0.6765,1.2647,1.9412,1.8235,0.4118,0.2353,5.9412,NaN,3.94375
2,Jesus Verdejo,South Florida,BE,27,72.0,96.2,21.8,45.7,47.98,2.1,...,-0.883163,0.6296,2.3333,2.9630,1.9630,0.4815,0.0000,12.1852,NaN,10.92680
3,Mike Hornbuckle,Pepperdine,WCC,30,44.5,97.7,16.0,53.6,53.69,4.1,...,-0.393459,0.7000,1.4333,2.1333,1.1000,0.5667,0.1333,4.9333,NaN,6.77427
4,Anthony Brown,Pacific,BW,33,56.2,96.5,22.0,52.8,54.31,8.3,...,-0.668318,1.4242,3.3030,4.7273,0.8485,0.4545,0.3333,7.5758,NaN,0.00000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61056,Trey Patterson,Villanova,BE,2,0.3,60.5,19.3,0.0,0.00,0.0,...,16.018200,0.0000,0.0000,0.0000,0.5000,0.5000,0.0000,0.0000,Pure PG,0.00000
61057,Stavros Polatoglou,Northwestern St.,Slnd,4,1.3,28.3,7.1,0.0,0.00,7.0,...,-4.993820,0.2500,0.0000,0.2500,0.0000,0.0000,0.0000,0.0000,C,0.00000
61058,Sandy Ryan,Tulane,Amer,1,0.1,0.0,0.0,0.0,0.00,0.0,...,-1.126810,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,PF/C,0.00000
61059,Ty Larson,Texas Tech,B12,1,0.1,0.0,0.0,0.0,0.00,0.0,...,-2.380750,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,PF/C,0.00000


In [7]:
#Creating a column called drafted with a 0(undrafted) and a 1(drafted)
cleaned_players_df["drafted"] = ""
results = []
for index, row in cleaned_players_df.iterrows():
  if(row["pick"] >= 1):
    results.append(1)
  else:
    results.append(0)

cleaned_players_df["drafted"] = results
cleaned_players_df

,player_name,team,conf,GP,Min_per,Ortg,usg,eFG,TS_per,ORB_per,...,oreb,dreb,treb,ast,stl,blk,pts,Unnamed: 64,Unnamed: 65,drafted
0,DeAndrae Ross,South Alabama,SB,26,29.5,97.3,16.6,42.5,44.43,1.6,...,0.1923,0.6154,0.8077,1.1923,0.3462,0.0385,3.8846,NaN,6.22026,0
1,Pooh Williams,Utah St.,WAC,34,60.9,108.3,14.9,52.4,54.48,3.8,...,0.6765,1.2647,1.9412,1.8235,0.4118,0.2353,5.9412,NaN,3.94375,0
2,Jesus Verdejo,South Florida,BE,27,72.0,96.2,21.8,45.7,47.98,2.1,...,0.6296,2.3333,2.9630,1.9630,0.4815,0.0000,12.1852,NaN,10.92680,0
3,Mike Hornbuckle,Pepperdine,WCC,30,44.5,97.7,16.0,53.6,53.69,4.1,...,0.7000,1.4333,2.1333,1.1000,0.5667,0.1333,4.9333,NaN,6.77427,0
4,Anthony Brown,Pacific,BW,33,56.2,96.5,22.0,52.8,54.31,8.3,...,1.4242,3.3030,4.7273,0.8485,0.4545,0.3333,7.5758,NaN,0.00000,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61056,Trey Patterson,Villanova,BE,2,0.3,60.5,19.3,0.0,0.00,0.0,...,0.0000,0.0000,0.0000,0.5000,0.5000,0.0000,0.0000,Pure PG,0.00000,0
61057,Stavros Polatoglou,Northwestern St.,Slnd,4,1.3,28.3,7.1,0.0,0.00,7.0,...,0.2500,0.0000,0.2500,0.0000,0.0000,0.0000,0.0000,C,0.00000,0
61058,Sandy Ryan,Tulane,Amer,1,0.1,0.0,0.0,0.0,0.00,0.0,...,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,PF/C,0.00000,0
61059,Ty Larson,Texas Tech,B12,1,0.1,0.0,0.0,0.0,0.00,0.0,...,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,PF/C,0.00000,0


In [8]:
#Checking to make sure that there are 1435 1's in the drafted column
count = 0
for value in cleaned_players_df["drafted"]:
  if(value == 1):
    count = count + 1

print(count)

1435


In [9]:
#Dropping the pick value
cleaned_players_df = cleaned_players_df.drop(columns=["pick", "Unnamed: 64", "Unnamed: 65"])
cleaned_players_df

,player_name,team,conf,GP,Min_per,Ortg,usg,eFG,TS_per,ORB_per,...,ogbpm,dgbpm,oreb,dreb,treb,ast,stl,blk,pts,drafted
0,DeAndrae Ross,South Alabama,SB,26,29.5,97.3,16.6,42.5,44.43,1.6,...,-2.781990,-1.941150,0.1923,0.6154,0.8077,1.1923,0.3462,0.0385,3.8846,0
1,Pooh Williams,Utah St.,WAC,34,60.9,108.3,14.9,52.4,54.48,3.8,...,-0.052263,-0.247934,0.6765,1.2647,1.9412,1.8235,0.4118,0.2353,5.9412,0
2,Jesus Verdejo,South Florida,BE,27,72.0,96.2,21.8,45.7,47.98,2.1,...,1.548230,-0.883163,0.6296,2.3333,2.9630,1.9630,0.4815,0.0000,12.1852,0
3,Mike Hornbuckle,Pepperdine,WCC,30,44.5,97.7,16.0,53.6,53.69,4.1,...,-0.342775,-0.393459,0.7000,1.4333,2.1333,1.1000,0.5667,0.1333,4.9333,0
4,Anthony Brown,Pacific,BW,33,56.2,96.5,22.0,52.8,54.31,8.3,...,-1.684860,-0.668318,1.4242,3.3030,4.7273,0.8485,0.4545,0.3333,7.5758,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61056,Trey Patterson,Villanova,BE,2,0.3,60.5,19.3,0.0,0.00,0.0,...,2.143290,16.018200,0.0000,0.0000,0.0000,0.5000,0.5000,0.0000,0.0000,0
61057,Stavros Polatoglou,Northwestern St.,Slnd,4,1.3,28.3,7.1,0.0,0.00,7.0,...,-7.242230,-4.993820,0.2500,0.0000,0.2500,0.0000,0.0000,0.0000,0.0000,0
61058,Sandy Ryan,Tulane,Amer,1,0.1,0.0,0.0,0.0,0.00,0.0,...,-3.524820,-1.126810,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0
61059,Ty Larson,Texas Tech,B12,1,0.1,0.0,0.0,0.0,0.00,0.0,...,-1.876040,-2.380750,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0


In [10]:
#Dropping null values
cleaned_players_df = cleaned_players_df.dropna()

In [11]:
#Checking to make sure that there are 1435 1's in the drafted column
count = 0
for value in cleaned_players_df["drafted"]:
  if(value == 1):
    count = count + 1

print(count)

1435


In [12]:
#Exporting the cleaned df to csv
cleaned_players_df.to_csv('/content/drive/MyDrive/Colab Notebooks/Project-4/Resources/Cleaned_df.csv', index=False)

In [13]:
#Checking that the csv loads correctly
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Project-4/Resources/Cleaned_df.csv')
df

,player_name,team,conf,GP,Min_per,Ortg,usg,eFG,TS_per,ORB_per,...,ogbpm,dgbpm,oreb,dreb,treb,ast,stl,blk,pts,drafted
0,DeAndrae Ross,South Alabama,SB,26,29.5,97.3,16.6,42.5,44.43,1.6,...,-2.781990,-1.941150,0.1923,0.6154,0.8077,1.1923,0.3462,0.0385,3.8846,0
1,Pooh Williams,Utah St.,WAC,34,60.9,108.3,14.9,52.4,54.48,3.8,...,-0.052263,-0.247934,0.6765,1.2647,1.9412,1.8235,0.4118,0.2353,5.9412,0
2,Jesus Verdejo,South Florida,BE,27,72.0,96.2,21.8,45.7,47.98,2.1,...,1.548230,-0.883163,0.6296,2.3333,2.9630,1.9630,0.4815,0.0000,12.1852,0
3,Mike Hornbuckle,Pepperdine,WCC,30,44.5,97.7,16.0,53.6,53.69,4.1,...,-0.342775,-0.393459,0.7000,1.4333,2.1333,1.1000,0.5667,0.1333,4.9333,0
4,Anthony Brown,Pacific,BW,33,56.2,96.5,22.0,52.8,54.31,8.3,...,-1.684860,-0.668318,1.4242,3.3030,4.7273,0.8485,0.4545,0.3333,7.5758,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60701,Abiram Frye,New Orleans,Slnd,1,0.2,0.0,0.0,0.0,0.00,0.0,...,-6.791950,1.201100,0.0000,1.0000,1.0000,0.0000,0.0000,0.0000,0.0000,0
60702,Kamron Battle,Northern Illinois,MAC,2,0.5,0.0,0.0,0.0,0.00,0.0,...,-14.203200,-6.921100,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0
60703,Trey Patterson,Villanova,BE,2,0.3,60.5,19.3,0.0,0.00,0.0,...,2.143290,16.018200,0.0000,0.0000,0.0000,0.5000,0.5000,0.0000,0.0000,0
60704,Stavros Polatoglou,Northwestern St.,Slnd,4,1.3,28.3,7.1,0.0,0.00,7.0,...,-7.242230,-4.993820,0.2500,0.0000,0.2500,0.0000,0.0000,0.0000,0.0000,0


In [14]:
#Checking the data types of each column
df.dtypes

player_name     object
team            object
conf            object
GP               int64
Min_per        float64
Ortg           float64
usg            float64
eFG            float64
TS_per         float64
ORB_per        float64
DRB_per        float64
AST_per        float64
TO_per         float64
FTM              int64
FTA              int64
FT_per         float64
twoPM            int64
twoPA            int64
twoP_per       float64
TPM              int64
TPA              int64
TP_per         float64
blk_per        float64
stl_per        float64
ftr            float64
yr              object
ht              object
porpag         float64
adjoe          float64
pfr            float64
year             int64
pid              int64
type            object
drtg           float64
adrtg          float64
dporpag        float64
stops          float64
bpm            float64
obpm           float64
dbpm           float64
gbpm           float64
mp             float64
ogbpm          float64
dgbpm      

In [15]:
#Reading csv into spark
basketball_data = spark.read.csv('/content/drive/MyDrive/Colab Notebooks/Project-4/Resources/Cleaned_df.csv',inferSchema=True, header=True)
basketball_data.show()

+---------------+------------------+----+---+-------+-----+----+----+------+-------+-------+-------+------+---+---+------+-----+-----+--------+---+---+------+-------+-------+-----+---+------+---------+--------+---+----+---+----+-------+-------+---------+--------+---------+---------+----------+---------+-------+----------+---------+------+------+------+------+------+------+-------+-------+
|    player_name|              team|conf| GP|Min_per| Ortg| usg| eFG|TS_per|ORB_per|DRB_per|AST_per|TO_per|FTM|FTA|FT_per|twoPM|twoPA|twoP_per|TPM|TPA|TP_per|blk_per|stl_per|  ftr| yr|    ht|   porpag|   adjoe|pfr|year|pid|type|   drtg|  adrtg|  dporpag|   stops|      bpm|     obpm|      dbpm|     gbpm|     mp|     ogbpm|    dgbpm|  oreb|  dreb|  treb|   ast|   stl|   blk|    pts|drafted|
+---------------+------------------+----+---+-------+-----+----+----+------+-------+-------+-------+------+---+---+------+-----+-----+--------+---+---+------+-------+-------+-----+---+------+---------+--------+---+--

In [16]:
#Creating a temporary view
basketball_data.createOrReplaceTempView('players')


In [17]:
#Query to get data for the model
query = """
SELECT  GP, MIN_per, mp, pts, eFG, TS_per, FTA, FTM, FT_per, twoPA, twoPM, twoP_per, TPA, TPM, TP_per, oreb, dreb, treb, ast, stl, blk, pfr, drafted
FROM players
"""
results = spark.sql(query)
results.show()

+---+-------+-------+-------+----+------+---+---+------+-----+-----+--------+---+---+------+------+------+------+------+------+------+---+-------+
| GP|MIN_per|     mp|    pts| eFG|TS_per|FTA|FTM|FT_per|twoPA|twoPM|twoP_per|TPA|TPM|TP_per|  oreb|  dreb|  treb|   ast|   stl|   blk|pfr|drafted|
+---+-------+-------+-------+----+------+---+---+------+-----+-----+--------+---+---+------+------+------+------+------+------+------+---+-------+
| 26|   29.5|14.5769| 3.8846|42.5| 44.43| 14| 10| 0.714|   68|   26|   0.382| 39| 13| 0.333|0.1923|0.6154|0.8077|1.1923|0.3462|0.0385|2.5|      0|
| 34|   60.9|24.5294| 5.9412|52.4| 54.48| 45| 30| 0.667|  113|   56|   0.496| 51| 20| 0.392|0.6765|1.2647|1.9412|1.8235|0.4118|0.2353|3.4|      0|
| 27|   72.0|33.1852|12.1852|45.7| 47.98| 67| 45| 0.672|  157|   67|   0.427|154| 50| 0.325|0.6296|2.3333| 2.963| 1.963|0.4815|   0.0|1.7|      0|
| 30|   44.5|17.9667| 4.9333|53.6| 53.69| 27| 14| 0.519|   63|   25|   0.397| 62| 28| 0.452|   0.7|1.4333|2.1333|   1.

In [18]:
#Converting spark query to pandas dataframe
df1 = results.toPandas()
df1.head()

,GP,MIN_per,mp,pts,eFG,TS_per,FTA,FTM,FT_per,twoPA,...,TPM,TP_per,oreb,dreb,treb,ast,stl,blk,pfr,drafted
0,26,29.5,14.5769,3.8846,42.5,44.43,14,10,0.714,68,...,13,0.333,0.1923,0.6154,0.8077,1.1923,0.3462,0.0385,2.5,0
1,34,60.9,24.5294,5.9412,52.4,54.48,45,30,0.667,113,...,20,0.392,0.6765,1.2647,1.9412,1.8235,0.4118,0.2353,3.4,0
2,27,72.0,33.1852,12.1852,45.7,47.98,67,45,0.672,157,...,50,0.325,0.6296,2.3333,2.9630,1.9630,0.4815,0.0000,1.7,0
3,30,44.5,17.9667,4.9333,53.6,53.69,27,14,0.519,63,...,28,0.452,0.7000,1.4333,2.1333,1.1000,0.5667,0.1333,2.0,0
4,33,56.2,22.9091,7.5758,52.8,54.31,114,64,0.561,176,...,0,0.000,1.4242,3.3030,4.7273,0.8485,0.4545,0.3333,3.8,0


In [19]:
#Importing whats needed for machine learning
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import tensorflow as tf

In [20]:
# Convert categorical data to numeric with `pd.get_dummies`
dummies = pd.get_dummies(df1, dtype=int)
dummies.head()

,GP,MIN_per,mp,pts,eFG,TS_per,FTA,FTM,FT_per,twoPA,...,TPM,TP_per,oreb,dreb,treb,ast,stl,blk,pfr,drafted
0,26,29.5,14.5769,3.8846,42.5,44.43,14,10,0.714,68,...,13,0.333,0.1923,0.6154,0.8077,1.1923,0.3462,0.0385,2.5,0
1,34,60.9,24.5294,5.9412,52.4,54.48,45,30,0.667,113,...,20,0.392,0.6765,1.2647,1.9412,1.8235,0.4118,0.2353,3.4,0
2,27,72.0,33.1852,12.1852,45.7,47.98,67,45,0.672,157,...,50,0.325,0.6296,2.3333,2.9630,1.9630,0.4815,0.0000,1.7,0
3,30,44.5,17.9667,4.9333,53.6,53.69,27,14,0.519,63,...,28,0.452,0.7000,1.4333,2.1333,1.1000,0.5667,0.1333,2.0,0
4,33,56.2,22.9091,7.5758,52.8,54.31,114,64,0.561,176,...,0,0.000,1.4242,3.3030,4.7273,0.8485,0.4545,0.3333,3.8,0


In [21]:
# Split our preprocessed data into our features and target arrays
y = dummies["drafted"]
X = dummies.drop(columns=["drafted"])

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, random_state=1)

In [22]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [23]:
len(X_train_scaled[0])

22

In [24]:
import tensorflow as tf
from imblearn.over_sampling import SMOTE
from sklearn.metrics import classification_report
import numpy as np

# Assuming X_train_scaled, X_test_scaled, y_train, and y_test are your data
# Apply SMOTE to the training data
smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X_train_scaled, y_train)

# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
input_features = len(X_resampled[0])
hidden_nodes_layer1 = 50
hidden_nodes_layer2 = 40
hidden_nodes_layer3 = 20

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer1,
        input_dim=input_features, activation="relu"))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Third hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Train the model on the resampled data
fit_model = nn.fit(X_resampled, y_resampled, epochs=10)

# Evaluate your model
predictions = nn.predict(X_test_scaled)
target_names = ["Undrafted", "Drafted"]
predictions = [round(i[0]) for i in predictions]
print(classification_report(y_test, predictions, target_names=target_names))

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 50)                1150      
                                                                 
 dense_1 (Dense)             (None, 40)                2040      
                                                                 
 dense_2 (Dense)             (None, 20)                820       
                                                                 
 dense_3 (Dense)             (None, 1)                 21        
                                                                 
Total params: 4031 (15.75 KB)
Trainable params: 4031 (15.75 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
Epoch 1/10
2779/2779 [==============================] - 6s 2ms/step - loss: 0.2930 - accuracy: 0.8770
Epoch 2/10
2779/2779 [==============================

In [46]:
new_draft = spark.read.csv('/content/drive/MyDrive/Colab Notebooks/Project-4/Resources/modified_data.csv',inferSchema=True, header=True)
new_draft.show()

+-------------------+----+----+-------+----+----+----+----+---+---+---+---+-----+-----+--------+---+---+------+---+---+------+------+------+------------+
|             Player|  GP|  mp|MIN_per| pts|oreb|dreb|treb|ast|stl|blk|pfr|twoPM|twoPA|twoP_per|TPM|TPA|TP_per|FTM|FTA|FT_per|TS_Per|   eFG|     ast_tov|
+-------------------+----+----+-------+----+----+----+----+---+---+---+---+-----+-----+--------+---+---+------+---+---+------+------+------+------------+
|    Sarr, Alexandre|24.0|17.2|    0.4| 9.7| 1.4| 3.0| 4.4|0.9|0.5|1.3|1.4|  3.1|  5.2|   0.605|0.6|2.0| 0.298|1.7|2.3| 0.714| 0.654|56.338|0.8181818182|
|      Topic, Nikola|12.0|33.8|    0.8|18.4| 0.8| 2.8| 3.6|7.1|1.0|0.2|3.8|  5.5|  8.3|   0.667|1.2|4.5| 0.259|3.9|4.6| 0.855| 0.698|57.031|     2.21875|
|     Sheppard, Reed|33.0|28.9|    0.7|12.5| 0.4| 3.8| 4.1|4.5|2.5|0.7|1.9|  2.0|  3.6|   0.555|2.3|4.4| 0.521|1.6|2.0| 0.831| 0.747|68.125|        2.25|
|    Castle, Stephon|34.0|27.0|    0.7|11.1| 1.7| 3.0| 4.7|2.9|0.8|0.5|2.4| 

In [47]:
new_draft.count()

118

In [48]:
#Creating a temporary view
new_draft.createOrReplaceTempView('draft')

In [67]:
query2 = """
SELECT  GP, MIN_per, mp, pts, eFG, TS_per, FTA, FTM, FT_per, twoPA, twoPM, twoP_per, TPA, TPM, TP_per, oreb, dreb, treb, ast, stl, blk, pfr
FROM draft
"""
results2 = spark.sql(query2)
results2.show()

+----+-------+----+----+------+------+---+---+------+-----+-----+--------+---+---+------+----+----+----+---+---+---+---+
|  GP|MIN_per|  mp| pts|   eFG|TS_per|FTA|FTM|FT_per|twoPA|twoPM|twoP_per|TPA|TPM|TP_per|oreb|dreb|treb|ast|stl|blk|pfr|
+----+-------+----+----+------+------+---+---+------+-----+-----+--------+---+---+------+----+----+----+---+---+---+---+
|24.0|    0.4|17.2| 9.7|56.338| 0.654|2.3|1.7| 0.714|  5.2|  3.1|   0.605|2.0|0.6| 0.298| 1.4| 3.0| 4.4|0.9|0.5|1.3|1.4|
|12.0|    0.8|33.8|18.4|57.031| 0.698|4.6|3.9| 0.855|  8.3|  5.5|   0.667|4.5|1.2| 0.259| 0.8| 2.8| 3.6|7.1|1.0|0.2|3.8|
|33.0|    0.7|28.9|12.5|68.125| 0.747|2.0|1.6| 0.831|  3.6|  2.0|   0.555|4.4|2.3| 0.521| 0.4| 3.8| 4.1|4.5|2.5|0.7|1.9|
|34.0|    0.7|27.0|11.1|50.588| 0.628|3.2|2.4| 0.755|  6.3|  3.4|   0.544|2.2|0.6| 0.267| 1.7| 3.0| 4.7|2.9|0.8|0.5|2.4|
|32.0|    0.6|22.0|10.1|  52.0| 0.645|3.1|2.3|  0.74|  4.1|  2.1|   0.511|3.4|1.2| 0.352| 1.2| 2.6| 3.8|0.9|0.8|0.3|2.4|
|35.0|    0.6|22.5|13.0|64.458| 

In [68]:
#Converting spark query to pandas dataframe
df2 = results2.toPandas()
df2

,GP,MIN_per,mp,pts,eFG,TS_per,FTA,FTM,FT_per,twoPA,...,TPA,TPM,TP_per,oreb,dreb,treb,ast,stl,blk,pfr
0,24.0,0.4,17.2,9.7,56.338,0.654,2.3,1.7,0.714,5.2,...,2.0,0.6,0.298,1.4,3.0,4.4,0.9,0.5,1.3,1.4
1,12.0,0.8,33.8,18.4,57.031,0.698,4.6,3.9,0.855,8.3,...,4.5,1.2,0.259,0.8,2.8,3.6,7.1,1.0,0.2,3.8
2,33.0,0.7,28.9,12.5,68.125,0.747,2.0,1.6,0.831,3.6,...,4.4,2.3,0.521,0.4,3.8,4.1,4.5,2.5,0.7,1.9
3,34.0,0.7,27.0,11.1,50.588,0.628,3.2,2.4,0.755,6.3,...,2.2,0.6,0.267,1.7,3.0,4.7,2.9,0.8,0.5,2.4
4,32.0,0.6,22.0,10.1,52.000,0.645,3.1,2.3,0.740,4.1,...,3.4,1.2,0.352,1.2,2.6,3.8,0.9,0.8,0.3,2.4
5,35.0,0.6,22.5,13.0,64.458,0.760,4.0,2.3,0.583,8.1,...,0.2,0.1,0.250,2.5,4.9,7.4,1.5,0.5,2.5,2.0
6,34.0,0.8,31.0,14.1,49.235,0.584,2.0,1.4,0.696,8.4,...,3.4,0.9,0.261,1.6,5.0,6.6,1.9,0.9,1.9,2.4
7,29.0,0.8,31.9,19.5,48.550,0.615,3.9,2.9,0.728,12.2,...,3.3,0.8,0.240,1.6,5.1,6.7,2.9,2.3,0.9,3.2
8,32.0,0.6,23.3,15.2,56.637,0.652,3.1,2.4,0.796,6.8,...,4.5,2.0,0.444,0.3,2.7,2.9,3.9,1.0,0.1,2.6
9,36.0,0.8,30.6,21.7,53.750,0.664,5.7,4.4,0.772,9.5,...,6.5,2.6,0.397,0.8,4.1,4.9,1.8,0.7,0.6,1.8


In [69]:
# Convert categorical data to numeric with `pd.get_dummies`
dummies2 = pd.get_dummies(df2, dtype=int)
dummies2

,GP,MIN_per,mp,pts,eFG,TS_per,FTA,FTM,FT_per,twoPA,...,TPA,TPM,TP_per,oreb,dreb,treb,ast,stl,blk,pfr
0,24.0,0.4,17.2,9.7,56.338,0.654,2.3,1.7,0.714,5.2,...,2.0,0.6,0.298,1.4,3.0,4.4,0.9,0.5,1.3,1.4
1,12.0,0.8,33.8,18.4,57.031,0.698,4.6,3.9,0.855,8.3,...,4.5,1.2,0.259,0.8,2.8,3.6,7.1,1.0,0.2,3.8
2,33.0,0.7,28.9,12.5,68.125,0.747,2.0,1.6,0.831,3.6,...,4.4,2.3,0.521,0.4,3.8,4.1,4.5,2.5,0.7,1.9
3,34.0,0.7,27.0,11.1,50.588,0.628,3.2,2.4,0.755,6.3,...,2.2,0.6,0.267,1.7,3.0,4.7,2.9,0.8,0.5,2.4
4,32.0,0.6,22.0,10.1,52.000,0.645,3.1,2.3,0.740,4.1,...,3.4,1.2,0.352,1.2,2.6,3.8,0.9,0.8,0.3,2.4
5,35.0,0.6,22.5,13.0,64.458,0.760,4.0,2.3,0.583,8.1,...,0.2,0.1,0.250,2.5,4.9,7.4,1.5,0.5,2.5,2.0
6,34.0,0.8,31.0,14.1,49.235,0.584,2.0,1.4,0.696,8.4,...,3.4,0.9,0.261,1.6,5.0,6.6,1.9,0.9,1.9,2.4
7,29.0,0.8,31.9,19.5,48.550,0.615,3.9,2.9,0.728,12.2,...,3.3,0.8,0.240,1.6,5.1,6.7,2.9,2.3,0.9,3.2
8,32.0,0.6,23.3,15.2,56.637,0.652,3.1,2.4,0.796,6.8,...,4.5,2.0,0.444,0.3,2.7,2.9,3.9,1.0,0.1,2.6
9,36.0,0.8,30.6,21.7,53.750,0.664,5.7,4.4,0.772,9.5,...,6.5,2.6,0.397,0.8,4.1,4.9,1.8,0.7,0.6,1.8


In [70]:
# Create a StandardScaler instances
scaler2 = StandardScaler()

# Fit the StandardScaler
X_scaler2 = scaler2.fit(dummies2)

# Scale the data
X_test_scaled2 = X_scaler2.transform(dummies2)

In [71]:
print("Length of X_test_scaled2:", len(X_test_scaled2))
print("Length of dummies2:", len(dummies2))


Length of X_test_scaled2: 118
Length of dummies2: 118


In [72]:
# Generate predictions
predictions2 = nn.predict(X_test_scaled2)
predictions2 = [round(i[0]) for i in predictions2]



4/4 [==============================] - 0s 4ms/step


In [73]:
print(predictions2)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [74]:
dummies2["Drafted"] = predictions2
dummies2

,GP,MIN_per,mp,pts,eFG,TS_per,FTA,FTM,FT_per,twoPA,...,TPM,TP_per,oreb,dreb,treb,ast,stl,blk,pfr,Drafted
0,24.0,0.4,17.2,9.7,56.338,0.654,2.3,1.7,0.714,5.2,...,0.6,0.298,1.4,3.0,4.4,0.9,0.5,1.3,1.4,0
1,12.0,0.8,33.8,18.4,57.031,0.698,4.6,3.9,0.855,8.3,...,1.2,0.259,0.8,2.8,3.6,7.1,1.0,0.2,3.8,0
2,33.0,0.7,28.9,12.5,68.125,0.747,2.0,1.6,0.831,3.6,...,2.3,0.521,0.4,3.8,4.1,4.5,2.5,0.7,1.9,0
3,34.0,0.7,27.0,11.1,50.588,0.628,3.2,2.4,0.755,6.3,...,0.6,0.267,1.7,3.0,4.7,2.9,0.8,0.5,2.4,0
4,32.0,0.6,22.0,10.1,52.000,0.645,3.1,2.3,0.740,4.1,...,1.2,0.352,1.2,2.6,3.8,0.9,0.8,0.3,2.4,0
5,35.0,0.6,22.5,13.0,64.458,0.760,4.0,2.3,0.583,8.1,...,0.1,0.250,2.5,4.9,7.4,1.5,0.5,2.5,2.0,0
6,34.0,0.8,31.0,14.1,49.235,0.584,2.0,1.4,0.696,8.4,...,0.9,0.261,1.6,5.0,6.6,1.9,0.9,1.9,2.4,0
7,29.0,0.8,31.9,19.5,48.550,0.615,3.9,2.9,0.728,12.2,...,0.8,0.240,1.6,5.1,6.7,2.9,2.3,0.9,3.2,0
8,32.0,0.6,23.3,15.2,56.637,0.652,3.1,2.4,0.796,6.8,...,2.0,0.444,0.3,2.7,2.9,3.9,1.0,0.1,2.6,0
9,36.0,0.8,30.6,21.7,53.750,0.664,5.7,4.4,0.772,9.5,...,2.6,0.397,0.8,4.1,4.9,1.8,0.7,0.6,1.8,0


In [75]:
query3 = """
SELECT  Player, GP, MIN_per, mp, pts, eFG, TS_per, FTA, FTM, FT_per, twoPA, twoPM, twoP_per, TPA, TPM, TP_per, oreb, dreb, treb, ast, ast_tov, stl, blk, pfr
FROM draft
"""
results3 = spark.sql(query3)
df3 = results3.toPandas()



dummies2["Player"] = df3["Player"]

In [76]:
pd.set_option('display.max_rows', 120)

# Display the first 118 rows of the DataFrame
dummies2.head(118)

,GP,MIN_per,mp,pts,eFG,TS_per,FTA,FTM,FT_per,twoPA,...,TP_per,oreb,dreb,treb,ast,stl,blk,pfr,Drafted,Player
0,24.0,0.4,17.2,9.7,56.338,0.654,2.3,1.7,0.714,5.2,...,0.298,1.4,3.0,4.4,0.9,0.5,1.3,1.4,0,"Sarr, Alexandre"
1,12.0,0.8,33.8,18.4,57.031,0.698,4.6,3.9,0.855,8.3,...,0.259,0.8,2.8,3.6,7.1,1.0,0.2,3.8,0,"Topic, Nikola"
2,33.0,0.7,28.9,12.5,68.125,0.747,2.0,1.6,0.831,3.6,...,0.521,0.4,3.8,4.1,4.5,2.5,0.7,1.9,0,"Sheppard, Reed"
3,34.0,0.7,27.0,11.1,50.588,0.628,3.2,2.4,0.755,6.3,...,0.267,1.7,3.0,4.7,2.9,0.8,0.5,2.4,0,"Castle, Stephon"
4,32.0,0.6,22.0,10.1,52.000,0.645,3.1,2.3,0.740,4.1,...,0.352,1.2,2.6,3.8,0.9,0.8,0.3,2.4,0,"Risacher, Zaccharie"
5,35.0,0.6,22.5,13.0,64.458,0.760,4.0,2.3,0.583,8.1,...,0.250,2.5,4.9,7.4,1.5,0.5,2.5,2.0,0,"Clingan, Donovan"
6,34.0,0.8,31.0,14.1,49.235,0.584,2.0,1.4,0.696,8.4,...,0.261,1.6,5.0,6.6,1.9,0.9,1.9,2.4,0,"Buzelis, Matas"
7,29.0,0.8,31.9,19.5,48.550,0.615,3.9,2.9,0.728,12.2,...,0.240,1.6,5.1,6.7,2.9,2.3,0.9,3.2,0,"Holland, Ron"
8,32.0,0.6,23.3,15.2,56.637,0.652,3.1,2.4,0.796,6.8,...,0.444,0.3,2.7,2.9,3.9,1.0,0.1,2.6,0,"Dillingham, Rob"
9,36.0,0.8,30.6,21.7,53.750,0.664,5.7,4.4,0.772,9.5,...,0.397,0.8,4.1,4.9,1.8,0.7,0.6,1.8,0,"Knecht, Dalton"


DRAFT ATTEMPT # 2

In [ ]:
#Checking that the csv loads correctly
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Project-4/Resources/cleaned_og.csv')
df

#Reading csv into spark
basketball_data = spark.read.csv('/content/drive/MyDrive/Colab Notebooks/Project-4/Resources/cleaned_og.csv',inferSchema=True, header=True)
basketball_data.show()
basketball_data = basketball_data.dropna()


In [ ]:
#Query to get data for the model
query = """
SELECT  GP, MIN, PTS, eFG, FTA, FTM, `FT%`, FGA, FGM, `FG%`, 3PA, 3PM, `3P%`, REB, AST, STL, BLK, drafted
FROM players
"""
results = spark.sql(query)
results.show()

In [ ]:
#Converting spark query to pandas dataframe
df1 = results.toPandas()
df1.head()

In [ ]:
#Importing whats needed for machine learning
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import tensorflow as tf

dummies = pd.get_dummies(df1, dtype=int)

# Split our preprocessed data into our features and target arrays
y = dummies["drafted"]
X = dummies.drop(columns=["drafted"])

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, random_state=1)

# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [ ]:
import tensorflow as tf
from imblearn.over_sampling import SMOTE
from sklearn.metrics import classification_report
import numpy as np

# Assuming X_train_scaled, X_test_scaled, y_train, and y_test are your data
# Apply SMOTE to the training data
smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X_train_scaled, y_train)

# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
input_features = len(X_resampled[0])
hidden_nodes_layer1 = 50
hidden_nodes_layer2 = 40
hidden_nodes_layer3 = 20

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer1,
        input_dim=input_features, activation="relu"))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Third hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Train the model on the resampled data
fit_model = nn.fit(X_resampled, y_resampled, epochs=10)

# Evaluate your model
predictions = nn.predict(X_test_scaled)
target_names = ["Undrafted", "Drafted"]
predictions = [round(i[0]) for i in predictions]
print(classification_report(y_test, predictions, target_names=target_names))

In [ ]:
new_draft = spark.read.csv('/content/drive/MyDrive/Colab Notebooks/Project-4/Resources/top750.csv',inferSchema=True, header=True)
new_draft.show()

In [ ]:
#Creating a temporary view
new_draft.createOrReplaceTempView('name')

query7 = """
SELECT  Name
FROM name
"""
results7 = spark.sql(query7)
results7.show()

df7 = results7.toPandas()
df7

In [ ]:
#Creating a temporary view
new_draft.createOrReplaceTempView('draft')

In [ ]:
#Query to get data for the model
query = """
SELECT  GP, MIN, PTS, eFG, FTA, FTM, `FT%`, FGA, FGM, `FG%`, 3PA, 3PM, `3P%`, REB, AST, STL, BLK,
FROM draft
"""
results2 = spark.sql(query)
results2.show()

In [ ]:
#Converting spark query to pandas dataframe
df2 = results2.toPandas()

# Convert categorical data to numeric with `pd.get_dummies`
dummies2 = pd.get_dummies(df2, dtype=int)
dummies2

In [ ]:
# Create a StandardScaler instances
scaler2 = StandardScaler()

# Fit the StandardScaler
X_scaler2 = scaler2.fit(dummies2)

# Scale the data
X_test_scaled2 = X_scaler2.transform(dummies2)

# Generate predictions
predictions2 = nn.predict(X_test_scaled2)
predictions2 = [round(i[0]) for i in predictions2]


In [ ]:
dummies2["Drafted"] = predictions2

filtered_dummies2 = dummies2[dummies2['Drafted'] == 1]
names_draft_1 = filtered_dummies2['Player']
print(names_draft_1)